# wikipedia 데이터 탐색
- `wikipedia_documents.json` 파일에 대한 탐색 내용입니다
- 살펴본 내용은 다음과 같습니다.

1. 데이터 중복 확인
    - document id에 중복 존재하는가? -> 중복 없음
    - text에 중복 존재하는가? -> 중복 있음 (7677개 text)
2. wiki 텍스트 단위 구성
    - 어떤 단위로 wiki 텍스트 구성된걸까? -> 제목을 기준으로 나뉜 듯 함
3. train/validation의 context와 wikipedia text의 관계
    - train/valiation 데이터셋은 모두 wikipedia에서 나온 passage들인가? -> 맞음
        - 따라서 텍스트에 대한 전처리 로직 만들어서, wikipedia, mrc(train, validation, test) text 데이터에 일괄적으로 전처리 수행하면 될 것으로 보임
4. wikipedia 문서 기준 전처리 할 내용 탐색
    1. 개행 (`\n`)
    2. 마크다운 문법 (`*`를 이용한 목록)
    3. 수식
    4. 미흡한 파싱 (`]]`)
    5. 미흡한 파싱 (문서 링크)
    6. 띄어쓰기 (붙여씀 / 2번 이상 띄어씀)

---

## 0. 데이터 준비

In [1]:
from datasets import load_from_disk
import pandas as pd
import json
from pprint import pprint

# # 각 cell에서 truncation 없이 모든 내용 출력
# pd.set_option('display.max_colwidth', None)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# wikipedia 데이터 불러오기
with open("../data/wikipedia_documents.json", "r") as f:
    wiki = json.load(f)

wiki_df = pd.DataFrame(wiki)
wiki_df = wiki_df.transpose()
wiki_df = wiki_df[['title','text','document_id']]  # 필요한 column만 남기기
wiki_df.head()

,title,text,document_id
0,나라 목록,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",0
1,나라 목록,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,1
2,백남준,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,2
3,아오조라 문고,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",3
4,아오조라 문고,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,4


In [3]:
# train, validation 데이터 불러오기 (wikipedia 데이터와의 일치 확인 목적)

# dataset을 pandas dataframe으로 변환하는 함수
def dataset_to_df(dataset_name):
    data_list = []

    # 필요한 항목 추출
    for item in dataset_name:
        data_list.append({
            'title': item['title'],
            'context': item['context'],
            'question': item['question'],
            'id': item['id'],
            'answer_start': item['answers']['answer_start'][0],
            'answer_text': item['answers']['text'][0],
            'document_id': item['document_id'],
        })

    # pandas dataframe으로 변환
    result_df = pd.DataFrame(data_list)
    return result_df


dataset = load_from_disk("../data/train_dataset/")
train_data, valid_data = dataset['train'], dataset['validation']
train_df = dataset_to_df(train_data)
valid_df = dataset_to_df(valid_data)

---

## 1. 데이터 중복 확인
- document id에 중복 존재하는가? -> 중복 없음
- text에 중복 존재하는가? -> 중복 있음 (7677개 text)
    - 이유 1. 같은 문서(동일한 title)가 여러 번 수집 (7586/7677건)
    - 이유 2. 동일한 문장이 여러 문서에 사용된 경우 (91/7677건)

In [4]:
# Q. document_id에 중복 존재하는가? -> 중복 없음
# A. 중복 없음. 다음 두 숫자 일치하므로.
print(f"전체 문서 개수: {wiki_df.shape[0]}")
print(f"unique한 document id 개수: {len(wiki_df.document_id.unique())}")

전체 문서 개수: 60613
unique한 document id 개수: 60613


In [5]:
# Q. text에 중복 존재하는가?
# A. 중복 있음. 7677개 text에서 서로 중복 확인됨

# Q. 그럼 왜 중복 발생하나?
# A-1. 같은 문서(동일한 title)가 여러 번 수집된 경우 (대부분의 경우: 7586/7677건)
# A-2. 동일한 문장이 여러 문서에 사용된 경우 (91/7677건)

# 중복된 데이터 찾기 (text를 기준으로)
duplicates = wiki_df[wiki_df.duplicated(subset=['text'], keep=False)]

# title이 동일한 경우의 중복 건수
same_title_duplicates = duplicates[duplicates.duplicated(subset=['text', 'title'], keep=False)]

# title이 다른 경우의 중복 건수
different_title_duplicates = duplicates[~duplicates.duplicated(subset=['text', 'title'], keep=False)]

# unique한 text
wiki_df_unique = wiki_df.drop_duplicates(subset=['text'])

# 결과 출력
print(f"전체 중복 행 수: {len(duplicates)}")
print(f"Title이 동일한 중복 건수: {len(same_title_duplicates)}")
print(f"Title이 다른 중복 건수: {len(different_title_duplicates)}")
print("-"*50)
print(f"전체 wiki 데이터 수: {len(wiki_df)}")
print(f"text 중복 제거한 wiki 데이터 수: {len(wiki_df_unique)}")

전체 중복 행 수: 7677
Title이 동일한 중복 건수: 7586
Title이 다른 중복 건수: 91
--------------------------------------------------
전체 wiki 데이터 수: 60613
text 중복 제거한 wiki 데이터 수: 56737


In [ ]:
# 같은 문서가 여러 번 수집되어 중복 발생한 예시
wiki_df[wiki_df['title']=='로봇 배제 표준']

,title,text,document_id
3859,로봇 배제 표준,"""Robots.txt""는 이 문서를 가리킵니다. 위키백과의 Robots.txt의 파...",3859
3860,로봇 배제 표준,"만약 모든 로봇에게 문서 접근을 허락하려면, robots.txt에 다음과 같이 입력...",3860
8413,로봇 배제 표준,"""Robots.txt""는 이 문서를 가리킵니다. 위키백과의 Robots.txt의 파...",8413
8414,로봇 배제 표준,"만약 모든 로봇에게 문서 접근을 허락하려면, robots.txt에 다음과 같이 입력...",8414


In [ ]:
wiki_df[wiki_df['title']=='아미타불']

,title,text,document_id
3743,아미타불,"아미타불(阿彌陀佛), 무량광불(無量光佛) 또는 무량수불(無量壽佛)은 서방 극락세계(...",3743
3744,아미타불,"""아미타불""이라는 낱말은 무량광(無量光: 헤아릴 수 없는 광명을 가진 것)의 뜻인 ...",3744
3745,아미타불,"과거에, 아미타불이 되기 이전인 법장보살 때에 처음 세운 원으로서, 《불설무량수경》...",3745
8297,아미타불,"아미타불(阿彌陀佛), 무량광불(無量光佛) 또는 무량수불(無量壽佛)은 서방 극락세계(...",8297
8298,아미타불,"""아미타불""이라는 낱말은 무량광(無量光: 헤아릴 수 없는 광명을 가진 것)의 뜻인 ...",8298
8299,아미타불,"과거에, 아미타불이 되기 이전인 법장보살 때에 처음 세운 원으로서, 《불설무량수경》...",8299


In [ ]:
# 동일한 문장이 여러 문서에 사용도니 경우 예시
wiki_df[wiki_df.text == "김정은의 집권 이후 조선민주주의인민공화국은 예산제에서 전면적으로 독립채산제로 설정을 하였으며 특히 평양 집적회로 공장과 평양 집적회로 시험 공장과 리철호 가 사업하는 기계 공장에도 전면적으로 독립채산제를 갖추어 나가기 시작하였다.\n\n2014년에는 기업이 가격을 제정할 수 있는 상품의 범위를 확대한 것으로 여기에는 대학에서 첨단기술제품을 수요자와 계약에 따라 생산하여 내화 및 외화로 판매하는 제품의 가격과 주문계약으로 생산한 제품 가운데 외화로 원가를 보상하여야 할 제품의 외화가격 등이 포함되었다.\n\n특히 반도체나 특히 소프트웨어등을 개발하여 판매한 수익을 자체로 예산에 포함시켜서 수익사업에 나설 수 있도록 전면적으로 허용을 하였으며 특히 국정가격의 판매를 전면적으로 없애고 자율적으로 시장에 의하여 조절할 수 있도록 허용을 하였다."]

,title,text,document_id
3669,조선련봉총회사,김정은의 집권 이후 조선민주주의인민공화국은 예산제에서 전면적으로 독립채산제로 설정을...,3669
8223,조선련봉총회사,김정은의 집권 이후 조선민주주의인민공화국은 예산제에서 전면적으로 독립채산제로 설정을...,8223
56098,리철호가사업하는기계공장,김정은의 집권 이후 조선민주주의인민공화국은 예산제에서 전면적으로 독립채산제로 설정을...,56098
56106,평양집적회로시험공장,김정은의 집권 이후 조선민주주의인민공화국은 예산제에서 전면적으로 독립채산제로 설정을...,56106
58404,111호마스크제작소,김정은의 집권 이후 조선민주주의인민공화국은 예산제에서 전면적으로 독립채산제로 설정을...,58404
58793,조선백송회사,김정은의 집권 이후 조선민주주의인민공화국은 예산제에서 전면적으로 독립채산제로 설정을...,58793


In [ ]:
wiki_df[wiki_df.text == '삼마지(,\n,\n,\n,\n,\n,\n,\n,\n)는\n설일체유부의 5위 75법에서 심소법(心所法: 46가지) 중\n대지법(大地法: 10가지) 가운데 하나이며,\n유식유가행파와 법상종의 5위 100법에서 심소법(心所法: 51가지) 중\n별경심소(別境心所: 5가지) 가운데 하나이다.pp=69-81loc="三摩地". 2012년 9월 22일에 확인\n삼마지(三摩地)는 심일경성(心一境性)의 마음작용이다. 즉 마음(6식 또는 8식, 즉 심왕, 즉 심법)을 평등(平等)하게 유지하여 즉 혼침(惛沈)과 도거(掉舉)를 멀리 떠난 상태에서 그것이 하나의 대상에 전념(專念, 專住)하게 하는 마음작용이다. 달리 말하면, 마음(6식 또는 8식, 즉 심왕, 즉 심법)을 한 곳에 모아 산란하지 않게 하는 마음작용이다. 마치 뱀이 죽통(竹筒)에 들어가면 바로 펴지듯이 마음(6식 또는 8식, 즉 심왕, 즉 심법)이 삼마지에 들면 산란되지 않고 한결같게 된다. 문자 그대로 번역하여 () 삼마제(三摩提) 삼마제(三摩帝)라 하기도 하며, 의역하여 선정(禪定)loc="禪定(선정)". 2013년 3월 2일에 확인|ps=]]() 등지(等持) 정정(正定) 정의(定意) 조직정(調直定) 정심행처(正心行處)라 하기도 한다.pp=69-81세친 지음, 현장 한역, 권오민 번역|p=163 / 1397loc="三摩地". 2012년 9월 22일에 확인loc="三摩地(삼마지)". 2012년 9월 22일에 확인 (참고: 삼매())']

,title,text,document_id
32724,삼마지,"삼마지(,\n,\n,\n,\n,\n,\n,\n,\n)는\n설일체유부의 5위 75법에...",32724
33936,불교 용어 목록 (삼),"삼마지(,\n,\n,\n,\n,\n,\n,\n,\n)는\n설일체유부의 5위 75법에...",33936


---


## 2. wiki 텍스트 단위 구성
- 어떤 단위로 wiki 텍스트 구성된걸까? -> 제목을 기준으로 나뉜 듯 함

In [10]:
# Q. 어떤 단위로 wiki 텍스트 구성된걸까?
# A. 중제목/소제목 등 제목을 기준으로 나눈 것으로 보임. 위키피디아 페이지에서 해당 문서 제목 검색해서 직접 대조해봄

# 예시 1: '양태 (문법)' 문서 (https://ko.wikipedia.org/wiki/%EC%96%91%ED%83%9C_(%EB%AC%B8%EB%B2%95)#cite_note-FOOTNOTEBybee1994176-179-35)
wiki_df[wiki_df['title']=='양태 (문법)']

,title,text,document_id
59541,양태 (문법),양태에 대한 가장 유명한 정의는 언어학자 라이언스(John Lyons)의 ‘문장이 ...,59541
59542,양태 (문법),영국의 언어학자 프랭크 팔머(Frank Robert Palmer)는 양태를 명제 양...,59542
59543,양태 (문법),증거성(evidentiality) 또는 증거 양태(evidential modalit...,59543


In [11]:
# 예시 2: '당나라의 이슬람교' 문서 (https://ko.wikipedia.org/wiki/%EB%8B%B9%EB%82%98%EB%9D%BC%EC%9D%98_%EC%9D%B4%EC%8A%AC%EB%9E%8C%EA%B5%90)
wiki_df[wiki_df['title']=='당나라의 이슬람교']

,title,text,document_id
59599,당나라의 이슬람교,"중국 무슬림의 전승에 따르면, 이슬람은 성사 무함마드가 사망한 지 20년도 채 되지...",59599
59600,당나라의 이슬람교,"하마다 하그라스(Hamada Hagras)는 ""이슬람은 당 고종 2년 여름인 서기 ...",59600
59601,당나라의 이슬람교,초기 중국 이슬람 건축물은 장안의 대청진사(大清真寺)로 742년(내부의 석판에 새겨...,59601
59602,당나라의 이슬람교,당대 이슬람은 무역과 상업에 관심이 많았고 이슬람의 확산에는 전혀 관심이 없던 아랍...,59602
59603,당나라의 이슬람교,"그러나 당대에는 양주 대학살(760년), 광주 대학살 등 무슬림에 대한 두 차례의 ...",59603


---

## 3. train/validation의 context와 wikipedia text의 관계
- train/valiation 데이터셋은 모두 wikipedia에서 나온 passage들인가? -> 맞음
    - 따라서 텍스트에 대한 전처리 로직 만들어서, wikipedia, mrc(train, validation, test) text 데이터에 일괄적으로 전처리 수행하면 될 것으로 보임

In [12]:
# Q. train/valiation 데이터셋은 모두 wikipedia에서 나온 passage들인가?
# A. 맞음

# wiki_texts를 hash set으로 변환
wiki_text_hashes = set(hash(text) for text in wiki_df['text'])  

# 각 context의 hash를 구하고, wiki_text_hashes에 있는지 확인
train_df['is_wikipedia'] = train_df['context'].apply(lambda context: hash(context) in wiki_text_hashes) 
train_df[~train_df['is_wikipedia']] # Wikipedia에 없는 passage만 필터링 (is_wikipedia가 False인 행들)

,title,context,question,id,answer_start,answer_text,document_id,is_wikipedia


In [13]:
valid_df['is_wikipedia'] = valid_df['context'].apply(lambda context: hash(context) in wiki_text_hashes) 
valid_df[~valid_df['is_wikipedia']]

,title,context,question,id,answer_start,answer_text,document_id,is_wikipedia


---

## 4. wikipedia 문서 기준 전처리 할 내용 탐색
- 랜덤하게 문서들 샘플링해서, 눈으로 훑으며 전처리할 내용 탐색함 (정량적 수치 근거는 없음)
1. 개행 (`\n`)
    - 예시: `개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n`
2. 마크다운 문법 (`*`를 이용한 목록)
    - 예시: `위 기준에 논거하여 이 목록은 다음 206개 국가를 포함하고 있다.\n* 일반 국제 승인을 받은 195개 자주 국가.\n** 유엔 가입 국가 193개\n바티칸 시국\n`
3. 수식
    - 예시: `:: x×(a y + b z) = a x × y + b x × z and (a y + b z) × x = a y × x + b z × x`
4. 미흡한 파싱 (`]]`)
    - 예시: `앞줄 왼쪽부터 퍼시벌 로웰, 홍영식, 민영익, 서광범, 중국인 통역 우리탕.]]\n`
5. 미흡한 파싱 (문서 링크)
    - 예시: `디오판토스의 산술 (책)|label=산술|en|Arithmetica(3세기) 등이 있다.`
6. 띄어쓰기 (붙여씀 / 2번 이상 띄어씀)
    - 예시: `홈팀이 흰색,원정팀이 유색 유니폼을 입은   1987년  본인(정주영)의 고향인 강원도를 연고지로 설정했다가  1990년`

In [14]:
# 1. 개행 (`\n`)
pprint(wiki_df[wiki_df.document_id == 0].text.iloc[0])

('이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n'
 '\n'
 '이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n'
 '\n'
 '# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 '
 '나라를 나열하고 있다.\n'
 '# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 '
 '나라를 나열하고 있다.\n'
 '\n'
 '두 목록은 모두 가나다 순이다.\n'
 '\n'
 '일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 '
 '과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 '
 '정보는 "국가" 문서에서 설명하고 있다.')


In [15]:
# 2. 마크다운 문법 (`*`를 이용한 목록)
pprint(wiki_df[wiki_df.document_id == 1].text.iloc[0])

('이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정에 따르면, 국가는 다음의 조건을 만족해야 한다.\n'
 '* (a) 영속적인 국민\n'
 '* (b) 일정한 영토\n'
 '* (c) 정부\n'
 '* (d) 타국과의 관계 참여 자격.\n'
 '특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 중요한 역할을 할 수 있다.  이 목록에 '
 '포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 자주적이고 독립적인 국가이다. 하지만 몬테비데오 협약 기준을 만족하는지의 '
 '여부는 많은 국가가 논쟁이 되고 있는 실정이다. 또한, 몬테비데오 협약 기준만이 국가 지위의 충분한 자격이든 아니든, 국제법의 견해 '
 '차이는 존재할 수 있다. 이 물음에 대한 다른 이론에 대한 고리는 아래에서 볼 수 있다.\n'
 '\n'
 '위 기준에 논거하여 이 목록은 다음 206개 국가를 포함하고 있다.\n'
 '* 일반 국제 승인을 받은 195개 자주 국가.\n'
 '** 유엔 가입 국가 193개\n'
 '** 성좌의 명칭으로 유엔에서 국제 승인을 받은 국가: 바티칸 시국\n'
 '** 팔레스타인지역에 위치하며 유엔 영구 옵서버 국가: 팔레스타인\n'
 '* 유엔의 가입국이 아니며, 일반 국제 승인을 받지 않은 11개 자주 국가.\n'
 '** 유엔 회원국으로부터 승인을 받은 8개 국가: 남오세티야, 니우에, 북키프로스 튀르크 공화국, 사하라 아랍 민주 공화국, 압하지야, '
 '중화민국, 코소보, 쿡 제도,\n'
 '** 유엔 비회원국으로부터 승인을 받은 2개 국가: 아르차흐 공화국, 트란스니스트리아\n'
 '** 어떤 나라에서도 승인 받지 않은 국가: 소말릴란드\n'
 '\n'
 '위 목록에 포함되지 않은 다음 국가는 몬테비데오 협약의 모든 조건을 만족하지 못하거나, 자주적이고 독립적임을 주장하지 않는 국가이다.\n'
 '* 남극 대륙 전체는 정부와 영속 인구가 없다. 7개 국가가 남극 

In [16]:
# 3. 수식
pprint(wiki_df[wiki_df.document_id == 32].text.iloc[0])

('7차원 벡터 공간의 벡터곱도 사원수의 방법을 팔원수에 적용하여 얻어질 수 있다.\n'
 '\n'
 '7차원 공간의 벡터곱은 다음과 같은 성질을 3차원 공간의 벡터곱과 공유한다.\n'
 '\n'
 '* 다음과 같은 의미에서 겹선형(bilinear)이다.\n'
 ':: x×(a y + b z) = a x × y + b x × z and (a y + b z) × x = a y × x + b z × '
 'x\n'
 '* 반가환성 (anti-commutative)\n'
 ':: x×y + y×x = 0\n'
 '* x와 y 모두에 수직\n'
 ':: x·(x×y) = y·(x×y) = 0\n'
 '* 야코비 항등식이 성립한다.\n'
 '::x×(y×z) + y×(z×x) + z×(x×y) = 0\n'
 '* ||x×y|| = ||x||||y||-(x·y)')


In [17]:
# 4. 미흡한 파싱 (`]]`)
pprint(wiki_df[wiki_df.document_id == 608].text.iloc[0])

('앞줄 왼쪽부터 퍼시벌 로웰, 홍영식, 민영익, 서광범, 중국인 통역 우리탕.]]\n'
 '보빙사(報聘使)는 1883년 조선에서 최초로 미국 등 서방 세계에 파견된 외교 사절단이다.\n'
 '\n'
 '1882년 조미 수호 통상 조약의 체결로 1883년 주한(駐韓) 공사 루시어스 푸트(Foote, L. H.)가 조선에 부임하였다. 이에 '
 '고종은 임오군란 이후 비대해진 청나라의 세력을 견제한다는 뜻에서 1883년 정사(正使)에 민영익(閔泳翊), 부사(副使)에 '
 '홍영식(洪英植), 서기관은 서광범(徐光範), 수행원은 변수(邊樹, 邊燧)·유길준(兪吉濬) 등 개화파 인사들을 대동시킨 친선 사절단을 서방 '
 '세계에 파견하였다. 사절단은 퍼시벌 로웰(Percival Lawrence Lowell)과 통역관 미야오카 츠네지로(宮岡恒次郎)의 인도 '
 '하에, 태평양을 건너 샌프란시스코에 도착하고 미대륙을 횡단한 다음 워싱턴을 거쳐 뉴욕에서 미국 대통령 체스터 A. 아서(Arthur, '
 'C. A.)와 2차례 회동하고 국서를 전하고 양국간의 우호와 교역에 관하여 논의하였다. 대통령과의 회동에서 보여준 조선식 전통 예법인 '
 '절은 소소한 충격을 주기도 했다. \n'
 '\n'
 '이어 세계박람회·시범농장·방직공장·의약제조회사·해군연병장·병원·전기회사·철도회사·소방서·육군사관학교 등 공공기관을 시찰하였다.\n'
 '특히, 워싱턴에서 내무성 교육국국장 이튼(Eaton, J.)을 방문하여 미국의 교육제도에 대하여 소개받았다. 뒤에 교육국사(敎育局史)와 '
 '연보를 기증받았다.\n'
 '그밖에 우편제도·전기시설·농업기술에 관심을 보였는데, 뒤에 우정국 설치, 경복궁의 전기설비, '
 '육영공원(育英公院)·농무목축시험장(農務牧畜試驗場) 등의 실현 계기가 되었다.\n'
 '\n'
 '이러한 인연으로 뒤에 주한미국공사 푸트를 통하여 육영공원 교사선발을 국장 이튼에게 의뢰, 뉴욕의 유니언 신학교(Union '
 'Theological Seminary)의 신학생 헐버트(

In [18]:
# 5. 미흡한 파싱 (문서 링크)
pprint(wiki_df[wiki_df.document_id == 4585].text.iloc[0])

('고대의 대수학에 대한 주요 저서로는 에우클레이데스의 원론(기원전 3세기)이나 구장산술(3세기), 디오판토스의 산술 '
 '(책)|label=산술|en|Arithmetica(3세기) 등이 있다. 9세기에 페르시아의 수학자 콰리즈미는 《약분·소거 '
 '계산론|en|The Compendious Book on Calculation by Completion and '
 'Balancing》(820년)를 통해 대수학을 하나의 독립적인 분야로 정립했다. 이 책은 1145년 체스터의 로버트|en|Robert '
 'of Chester가 《알게브라와 알무카발라의 서(書)》(Liber algebrae et almucabala)란 제목으로 라틴어로 번역한 '
 '뒤 다섯 세기에 걸쳐서 유럽의 대학에서 사용되었다. 여기서 "알게브라"(algebra)와 "알무카발라"(almucabala)는 해당하는 '
 '아랍어 단어를 음역한 것이다. 또한 콰리즈미의 저서인 "인도 수의 계산법"이 라틴어로 번역되면서 2차 방정식, 사칙연산, 십진법, 0 '
 '등의 개념이 소개되었다.\n'
 '\n'
 '19세기 이후에는 에바리스트 갈루아가 대수 방정식을 연구하기 위해서 군이라는 대수적 구조를 도입하였고, 조지 불은 논리학을 연구하기 '
 '위해서 불 대수라는 대수적 구조를 정의하였다. 이후 현대 수학에서는 다비트 힐베르트의 공리 주의나 니콜라 부르바키 스타일에서 찾아볼 수 '
 '있듯이, 고전적인 대수학에서 상당히 거리가 추상화되어 있으며, 방정식의 해법은 "방정식론"(대수방정식론)이라는 대수학의 일부분에 '
 '불과하다.')


In [19]:
# 6. 띄어쓰기 (붙여씀 / 2번 이상 띄어씀)
pprint(wiki_df[wiki_df.document_id == 39].text.iloc[0])

('1915년 강원도 통천군 답전면 아산리 (현 조선민주주의인민공화국 강원도 통천군 로상리)에서 아버지 정봉식과 어머니 한성실 사이에서 6남 '
 '2녀 중 장남으로 태어났다.  아산(峨山)이라는 그의 아호는 자신의 출생지 옛 지명에서 따온 것이다. 1930년 통천 송전소학교를 '
 '졸업하였고 그와 함께한 동창생은 27명이며 정주영의 최종 학력은 소학교(초등학교) 졸업 이 유일하다. \n'
 '상황이 이렇다 보니 프로축구 현대 호랑이는  홈 앤 어웨이 제도로 바뀐 한편 주말 2연전이 새롭게 도입된  동시에 프로야구의 선례를 본따 '
 '홈팀이 흰색,원정팀이 유색 유니폼을 입은   1987년  본인(정주영)의 고향인 강원도를 연고지로 설정했다가  1990년 울산으로 '
 '연고지를 옮겼으며  주말 2연전 제도는 선수 부상 등의  문제점이 생기자 1년 만에 폐지됐고  다음 해부터 홈팀은 유색, 원정팀은 흰색 '
 '유니폼을 입는 것으로 변경됐다.')
